In [ ]:
# NTRUEncrypt

# a post-quantum cryptosystem - alternative to RSA or ECC
# is reasonably well-studied and has so far managed to resist all attempts at cryptanalysis
# as such can be seen as a "conservative choice" for an RSA replacement
# however, the difficulty of breaking NTRU is only "strongly connected" to lattice problems
# compared to other cryptosystems, where the breaking of the cryptosystem can actually be reduced to lattice problems
# but may not need to be a problem - compare with the connection of breaking RSA and integer factorization

# some suplementary materials: shrek.unideb.hu/~tengely/crypto/section-8.html
# Mr. Snail slides: https://www.slideshare.net/44Con/how-to-explain-postquantum-cryptography-to-a-middle-school-student-klaus-schmeh-44con-2018

# recommended reading for credit project: https://eprint.iacr.org/2017/481.pdf

In [ ]:
### sage does not internaly use symmetric representation...
def cmap(t,p):
    if (ZZ(t)%p)>(p//2):
        return ((ZZ(t)%p)-p)
    else:
        return ZZ(t)%p

# the cryptosystem has multiple parameters - N,p,q - the maximal degree of polynomials, a small modulus and a large modulus respectively

# these are just toy parameters...
N = 7 # this should be a much larger prime - for 128 bit security approx 600
p = 3 # this is actually used in practice
q = 41 # this is usually a large power of 2 e.g. 2048

# standard computations run in the ring Z[X]/(X^N-1)
R.<x> = ZZ[]
Rp.<a> = Zmod(p)[]
Rq.<b> = Zmod(q)[]
#S = R.quotient(a^N - 1, 'x'); x = S.gen()
  
# this thing needs to be invertible, so in practice, we should check it 
f = R.random_element((N//2,N-1),-1,2)


# calculating modular inverses
fp = Rp(f).inverse_mod(Rp(x^N-1))
fq = Rq(f).inverse_mod(Rq(x^N-1))

# rewriting polynomials into symmetric representation
fp=R([cmap(k,p) for k in fp.list()])
fq=R([cmap(k,q) for k in fq.list()])

g = R.random_element((N//2,N-1),-1,2)

# f, fp, fq and g are all parts of private key

# h is the public key
h = (p*fq*g) % q

print(f)
print(g)

-x^3 + x^2 - x - 1
-x^4 - x^3 + x^2 + 1


In [ ]:
# If we want to encrypt a message we need to convert it to polynomial of degree less than N and with coefficients (-1,0,1)
m = [1,0,1,1,0,0,1]

m = R(m)
print (m)

# then we need a small error term - polynomial with small coefficients
e = R.random_element((N//2,N),-1,2)

c = h*e+m
c %= x^N-1
c %= q
print (c)

x^6 + x^3 + x^2 + 1
5*x^6 - 10*x^5 - 9*x^4 - 8*x^3 + 8*x^2 + 25*x + 34


In [ ]:
# To decrypt the message we need to get rid of the error term

d = (f*c) % (x^N-1)
d %= q
d = R([cmap(k,q) for k in d.list()])

# If we were to rewrite, this means d = f*(h*e+m) =f*h*e+f*m = f*m + f*e*(p*fq*g) =f*m + p*e*g
# to get rid of p*e*g, we simply compute d mod p

d = d%p
d = R([cmap(k,p) for k in d.list()])

# as we are now left with f*m we may multiply by fp

d *= fp
d %= x^N-1
d %= 3
d = R([cmap(k,p) for k in d.list()])
print (d.list())
print (m.list())


[1, 0, 1, 1, 0, 0, 1]
[1, 0, 1, 1, 0, 0, 1]


In [ ]:
# The connection with lattices:
# Consider the following structure:
# {(a,b) \in (Zq/(x^N-1))^2 : a*h = b }
# It is a lattice!!!
# (f, p*g) is a vector in this lattice
# q is much bigger than p -- (f, p*g) is a short vector
# therefore if we can find short vectors, we can break NTRU!
# however, the reverse implication is not known to hold

In [ ]:
# We can take a look at LLL's performance
M = matrix(2*N)
for i in [0..N-1]: M[i,i] = 1
for i in [N..2*N-1]: M[i,i] = q
for i in [0..N-1]:
    for j in [0..N-1]:
        M[i+N,j] = ((R(GF(q)(1/p)*h)*x^i)%(x^N-1))[j]
pretty_print(M)
pretty_print(M.transpose().LLL())

# We will be looking for these vectors
v1 = f.coefficients(sparse=False)+[0] * (N - len(f.coefficients(sparse=False)))
print(f)
v2 = g.coefficients(sparse=False)+[0] * (N - len(g.coefficients(sparse=False)))
print(g)
print(list(reversed(v1)))
print(list(reversed(v2)))
print(list(reversed(v1))+list(reversed(v2)))

[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  1  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  1  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0  0]
[49  5 72 49 19 40 12 41  0  0  0  0  0  0]
[12 49  5 72 49 19 40  0 41  0  0  0  0  0]
[40 12 49  5 72 49 19  0  0 41  0  0  0  0]
[19 40 12 49  5 72 49  0  0  0 41  0  0  0]
[49 19 40 12 49  5 72  0  0  0  0 41  0  0]
[72 49 19 40 12 49  5  0  0  0  0  0 41  0]
[ 5 72 49 19 40 12 49  0  0  0  0  0  0 41]

[ -1  -1  -1  -1  -1  -1  -1   0   0   0   0   0   0   0]
[ -1   1  -1  -1   0   0   0  -1   1   0   1   0   0  -1]
[  1  -1  -1   0   0   0  -1   1   0   1   0   0  -1  -1]
[ -1  -1   0   0   0  -1   1   0   1   0   0  -1  -1   1]
[  0   0  -1   1  -1  -1   0   0  -1  -1   1   0   1   0]
[  0   0   0  -1   1  -1  -1   0   0  -1  -1   1   0   1]
[  0  -1   1  -1  -1   0   0  -1  -1   1   0   1   0   0]
[ -1  -4   8   6   4  -5  -9  -5   5   1   4  -5   5  -5]
[  1   4  -3   5  -9  -1   6   0   5  -2  -9  10  -3  -1]
[  4  -5   9   1  -5  -2  -3   1   9 -10   4   2  -1  -5]
[ -6  -4   5   9   1   4  -8  -4   5  -5   5   5  -5  -1]
[ -5  -6  -4  -1  -1   5   9   5   1  -6  -5  11   0  -6]
[ -4   5   9   1   4  -8  -6   5  -5   5   5  -5  -1  -4]
[  5   1   9  -4   1  -4  -5   9   8   4  11  -1   6   4]

-x^3 + x^2 - x - 1
-x^4 - x^3 + x^2 + 1
[0, 0, 0, -1, 1, -1, -1]
[0, 0, -1, -1, 1, 0, 1]
[0, 0, 0, -1, 1, -1, -1, 0, 0, -1, -1, 1, 0, 1]


In [ ]:
# And now something a bit different - a chosen ciphertext attack
# (for simplicity, assume that f*h = g, and not f*h = p*g)

# We can try to break the scheme by fabricating our own messages
# we shall choose the form c = e*h+e, such that e == 0 mod p, c < q/2 < 2c

# let us try to "decrypt the message":

# f*c = f*e*h+f*e = e*g + e*f mod q

# that is f*c = e*g + e*f + q*K, where K is such polynomial, that is -1 if both f and g are -1, and 1 if both are 1
# (and the rest of coefficients is zero)

# if we reduce modulo p, we get f*c = q*K mod p, or c = q*fp*K mod p
# rewrite a bit more to get f = q*K*(c^-1) mod p

# if f and g are sparse, K is likely to have a small number of coefficients - we can bruteforce
